# Analyzing Assessed Value of the Downtown Mall

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import requests
import folium
from folium import plugins
import geojson

In [2]:
#Import .xls retrieved from the GIS Viewer into pandas

#Path to the .xls retrieve from the GIS Viewer
f = r'./data/mall_parcels_no_ends.xls'
#Create a dataframe that reads the .xls file
df = pd.read_excel(f)
#Identify all rows in df where MULTIPIN column is not equal to 1
not_multipin = df['MULTIPIN'] != 1
#Create a new dataframe that only contains the rows identified in not_multipin
df = df[not_multipin]

In [3]:
## Create Query URL's for open portal data based on .xls data

formatted_gpins = [str(x) for x in df['GPIN'].unique()]
formatted_gpins = formatted_gpins
formatted_gpins = ','.join(formatted_gpins)

# Url for Parcel Area Boundary geojson data
parcel_url = f'https://opendata.arcgis.com/datasets/320d465ddf0d498796da6491e21f6dde_43.geojson?where=GPIN%20in%20({formatted_gpins})'

formatted_pins = [f'%27{x}%27' for x in df['PIN'].unique()]
formatted_pins = formatted_pins
formatted_pins = ','.join(formatted_pins)

assessment_url = f"https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_2/MapServer/2/query?where=UPPER(ParcelNumber)%20in%20({formatted_pins})%20&outFields=ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear&outSR=4326&f=json"
assessment_request = requests.get(assessment_url)
assessment_json = assessment_request.json()

In [4]:
# Create data series based on features and combine data frames into a single df
assessment_df = pd.DataFrame(assessment_json['features'])
# Create a single data frame based on combined series data
assessments = pd.DataFrame([x for x in assessment_df['attributes']], dtype = 'object')
assessments = assessments.astype({'TaxYear': 'int64', 'ImprovementValue': 'int64','LandValue': 'int64','TotalValue': 'int64', 'ParcelNumber': 'str'})
assessments = assessments.drop(['ImprovementValue', 'LandValue'], axis=1)
#----------------
#Create data frame from df that holds PIN and GPIN.
df_key = pd.DataFrame(df[['PIN','GPIN']], dtype = 'str')
#----------------
# Merge df_key with assessments
d = pd.merge(assessments, df_key, how='inner', left_on=['ParcelNumber'], right_on=['PIN'])

In [5]:
bins = [0, 100000, 250000, 500000, 1000000, 2000000, 5000000, 10000000, 25000000, 50000000]
bin_color = ['#f7fcfd', '#EEF9FB', '#e5f5f9','#ccece6','#99d8c9','#66c2a4','#41ae76','#238b45','#006d2c','#00441b']
d.head()

t = d.drop(['ParcelNumber','PIN'], axis = 1)
t.head()
lol = t.values.tolist()
lol = [[x[2],f'{x[0]}-01-01', x[1]] for x in lol]
lol
for row in lol:
    if row[2] in range(bins[0],bins[1]+1):
        row[2] = bin_color[0]
    elif row[2] in range(bins[1],bins[2]+1):
        row[2] = bin_color[1]
    elif row[2] in range(bins[2],bins[3]+1):
        row[2] = bin_color[2]
    elif row[2] in range(bins[3],bins[4]+1):
        row[2] = bin_color[3]     
    elif row[2] in range(bins[4],bins[5]+1):
        row[2] = bin_color[4]
    elif row[2] in range(bins[5],bins[6]+1):
        row[2] = bin_color[5]
    elif row[2] in range(bins[6],bins[7]+1):
        row[2] = bin_color[6]
    elif row[2] in range(bins[7],bins[8]+1):
        row[2] = bin_color[7]
    elif row[2] in range(bins[8],bins[9]+1):
        row[2] = bin_color[8]
    elif row[2] in range(bins[9],bins[10]+1):
        row[2] = bin_color[9]

lol

[['6861', '2004-01-01', '#66c2a4'],
 ['6861', '2003-01-01', '#66c2a4'],
 ['6861', '2002-01-01', '#66c2a4'],
 ['6861', '2001-01-01', '#66c2a4'],
 ['6861', '2000-01-01', '#66c2a4'],
 ['6861', '1999-01-01', '#66c2a4'],
 ['6861', '1998-01-01', '#66c2a4'],
 ['6861', '1997-01-01', '#66c2a4'],
 ['6861', '2019-01-01', '#41ae76'],
 ['6861', '2018-01-01', '#41ae76'],
 ['6861', '2017-01-01', '#41ae76'],
 ['6861', '2016-01-01', '#66c2a4'],
 ['6861', '2015-01-01', '#66c2a4'],
 ['6861', '2014-01-01', '#66c2a4'],
 ['6861', '2013-01-01', '#66c2a4'],
 ['6861', '2012-01-01', '#66c2a4'],
 ['6861', '2011-01-01', '#66c2a4'],
 ['6861', '2010-01-01', '#41ae76'],
 ['6861', '2009-01-01', '#41ae76'],
 ['6861', '2008-01-01', '#41ae76'],
 ['6861', '2007-01-01', '#66c2a4'],
 ['6861', '2006-01-01', '#66c2a4'],
 ['6861', '2005-01-01', '#66c2a4'],
 ['6869', '2019-01-01', '#41ae76'],
 ['6869', '2018-01-01', '#41ae76'],
 ['6869', '2017-01-01', '#41ae76'],
 ['6869', '2016-01-01', '#41ae76'],
 ['6869', '2015-01-01', '#66

In [6]:
styledict = {
    'GPIN1': {'2017-1-1': {'color': 'ffffff'},'2017-1-2': {'color': 'fffff0'}},

    'GPIN2': {'2017-1-1': {'color': 'ffffff'},'2017-1-2': {'color': 'fffff0'}}}

In [7]:
df_lol = pd.DataFrame(lol, columns = ['GPIN', 'Year', 'Color'])
df_lol.head()

,GPIN,Year,Color
0,6861,2004-01-01,#66c2a4
1,6861,2003-01-01,#66c2a4
2,6861,2002-01-01,#66c2a4
3,6861,2001-01-01,#66c2a4
4,6861,2000-01-01,#66c2a4


In [8]:
styledata = {}

time_index = df_lol['Year'].unique()
print(time_index)
for gpin in df['GPIN'].unique():
    dfx = pd.DataFrame(df_lol[df_lol['GPIN'] == str(gpin)])
    dfx = pd.DataFrame({'color': dfx['Color']} ,index=time_index)
    print(dfx)
    #styledata[gpin] = dfx    
styledata


['2004-01-01' '2003-01-01' '2002-01-01' '2001-01-01' '2000-01-01'
 '1999-01-01' '1998-01-01' '1997-01-01' '2019-01-01' '2018-01-01'
 '2017-01-01' '2016-01-01' '2015-01-01' '2014-01-01' '2013-01-01'
 '2012-01-01' '2011-01-01' '2010-01-01' '2009-01-01' '2008-01-01'
 '2007-01-01' '2006-01-01' '2005-01-01']
           color
2004-01-01   NaN
2003-01-01   NaN
2002-01-01   NaN
2001-01-01   NaN
2000-01-01   NaN
1999-01-01   NaN
1998-01-01   NaN
1997-01-01   NaN
2019-01-01   NaN
2018-01-01   NaN
2017-01-01   NaN
2016-01-01   NaN
2015-01-01   NaN
2014-01-01   NaN
2013-01-01   NaN
2012-01-01   NaN
2011-01-01   NaN
2010-01-01   NaN
2009-01-01   NaN
2008-01-01   NaN
2007-01-01   NaN
2006-01-01   NaN
2005-01-01   NaN
           color
2004-01-01   NaN
2003-01-01   NaN
2002-01-01   NaN
2001-01-01   NaN
2000-01-01   NaN
1999-01-01   NaN
1998-01-01   NaN
1997-01-01   NaN
2019-01-01   NaN
2018-01-01   NaN
2017-01-01   NaN
2016-01-01   NaN
2015-01-01   NaN
2014-01-01   NaN
2013-01-01   NaN
2012-01-01   Na

           color
2004-01-01   NaN
2003-01-01   NaN
2002-01-01   NaN
2001-01-01   NaN
2000-01-01   NaN
1999-01-01   NaN
1998-01-01   NaN
1997-01-01   NaN
2019-01-01   NaN
2018-01-01   NaN
2017-01-01   NaN
2016-01-01   NaN
2015-01-01   NaN
2014-01-01   NaN
2013-01-01   NaN
2012-01-01   NaN
2011-01-01   NaN
2010-01-01   NaN
2009-01-01   NaN
2008-01-01   NaN
2007-01-01   NaN
2006-01-01   NaN
2005-01-01   NaN
           color
2004-01-01   NaN
2003-01-01   NaN
2002-01-01   NaN
2001-01-01   NaN
2000-01-01   NaN
1999-01-01   NaN
1998-01-01   NaN
1997-01-01   NaN
2019-01-01   NaN
2018-01-01   NaN
2017-01-01   NaN
2016-01-01   NaN
2015-01-01   NaN
2014-01-01   NaN
2013-01-01   NaN
2012-01-01   NaN
2011-01-01   NaN
2010-01-01   NaN
2009-01-01   NaN
2008-01-01   NaN
2007-01-01   NaN
2006-01-01   NaN
2005-01-01   NaN
           color
2004-01-01   NaN
2003-01-01   NaN
2002-01-01   NaN
2001-01-01   NaN
2000-01-01   NaN
1999-01-01   NaN
1998-01-01   NaN
1997-01-01   NaN
2019-01-01   NaN
2018-01-01   N

           color
2004-01-01   NaN
2003-01-01   NaN
2002-01-01   NaN
2001-01-01   NaN
2000-01-01   NaN
1999-01-01   NaN
1998-01-01   NaN
1997-01-01   NaN
2019-01-01   NaN
2018-01-01   NaN
2017-01-01   NaN
2016-01-01   NaN
2015-01-01   NaN
2014-01-01   NaN
2013-01-01   NaN
2012-01-01   NaN
2011-01-01   NaN
2010-01-01   NaN
2009-01-01   NaN
2008-01-01   NaN
2007-01-01   NaN
2006-01-01   NaN
2005-01-01   NaN
           color
2004-01-01   NaN
2003-01-01   NaN
2002-01-01   NaN
2001-01-01   NaN
2000-01-01   NaN
1999-01-01   NaN
1998-01-01   NaN
1997-01-01   NaN
2019-01-01   NaN
2018-01-01   NaN
2017-01-01   NaN
2016-01-01   NaN
2015-01-01   NaN
2014-01-01   NaN
2013-01-01   NaN
2012-01-01   NaN
2011-01-01   NaN
2010-01-01   NaN
2009-01-01   NaN
2008-01-01   NaN
2007-01-01   NaN
2006-01-01   NaN
2005-01-01   NaN
           color
2004-01-01   NaN
2003-01-01   NaN
2002-01-01   NaN
2001-01-01   NaN
2000-01-01   NaN
1999-01-01   NaN
1998-01-01   NaN
1997-01-01   NaN
2019-01-01   NaN
2018-01-01   N

2005-01-01   NaN
           color
2004-01-01   NaN
2003-01-01   NaN
2002-01-01   NaN
2001-01-01   NaN
2000-01-01   NaN
1999-01-01   NaN
1998-01-01   NaN
1997-01-01   NaN
2019-01-01   NaN
2018-01-01   NaN
2017-01-01   NaN
2016-01-01   NaN
2015-01-01   NaN
2014-01-01   NaN
2013-01-01   NaN
2012-01-01   NaN
2011-01-01   NaN
2010-01-01   NaN
2009-01-01   NaN
2008-01-01   NaN
2007-01-01   NaN
2006-01-01   NaN
2005-01-01   NaN
           color
2004-01-01   NaN
2003-01-01   NaN
2002-01-01   NaN
2001-01-01   NaN
2000-01-01   NaN
1999-01-01   NaN
1998-01-01   NaN
1997-01-01   NaN
2019-01-01   NaN
2018-01-01   NaN
2017-01-01   NaN
2016-01-01   NaN
2015-01-01   NaN
2014-01-01   NaN
2013-01-01   NaN
2012-01-01   NaN
2011-01-01   NaN
2010-01-01   NaN
2009-01-01   NaN
2008-01-01   NaN
2007-01-01   NaN
2006-01-01   NaN
2005-01-01   NaN
           color
2004-01-01   NaN
2003-01-01   NaN
2002-01-01   NaN
2001-01-01   NaN
2000-01-01   NaN
1999-01-01   NaN
1998-01-01   NaN
1997-01-01   NaN
2019-01-01   N

{}

In [9]:
styledata = {}

for gpin in df['GPIN'].unique():
    dfx = pd.DataFrame({'color': df_lol['color']['GPIN'] = gpin} ,index=gpin)
    styledata[gpin] = dfx
styledata

#From TimeSliderChoropleth, styledata is a dict, data is DataFrame
styledict = {
    str(country): data.to_dict(orient='index') for
    country, data in styledata.items()
}
# End

for line in lol:
    if line[0] in styledict:
        # append the new number to the existing array at this slot
        styledict[line[0]].append({line[1]: {'color': line[2]}})
    else:
        # create a new array in this slot
        styledict[line[0]] = [{line[1]: {'color': line[2]}}]
styledict


SyntaxError: invalid syntax (<ipython-input-9-45790da41da6>, line 4)

In [ ]:
styledict = {}

#for x in lol:
#    print({x[0]: {x[1]: {'color': x[2]}}})
    
for x in lol:
    styledict.update({x[0]: {x[1]: {'color': x[2]}}})
styledict

In [ ]:
m = folium.Map(location=[38.03090, -78.48044], zoom_start=17, tiles='OpenStreetMap')
folium.plugins.TimeSliderChoropleth(
    parcel_url,
    styledict = styledict, 
    name='Time Slider Demo', 
    overlay=True, 
    control=True, 
    show=True)
m

In [ ]:
#Store year_min, max, and a generator of years into variables
year_min = int(d['TaxYear'].min())
year_max = int(d['TaxYear'].max())
years = (x for x in range(year_min, year_max+1))

In [ ]:
def all_values_by_year(year):
    all_values_df = d[d['TaxYear'] == year]
    all_values_df = all_values_df.astype({'GPIN': 'int', 'TotalValue': 'int'})
    return all_values_df

In [ ]:
## Map assessed value by year
def choropleth_assessed_val_by_year(year):
    m = folium.Map(location=[38.03090, -78.48044], zoom_start=17, tiles='OpenStreetMap')
    folium.Choropleth(
        geo_data = parcel_url,
        data=all_values_by_year(year)[['GPIN', 'TotalValue']],
        columns=['GPIN', 'TotalValue'], 
        key_on='feature.properties.GPIN', 
        bins=[0, 250000, 500000, 1000000, 2000000, 5000000, 10000000, 16000000],
        fill_color= 'YlOrRd',
        nan_fill_color='black', 
        fill_opacity=0.9, 
        nan_fill_opacity=1, 
        line_color='black', 
        line_weight=1, 
        line_opacity=1, 
        name='Assessed Value',
        legend_name= f'Total Assessed Value {year}'
    ).add_to(m)
    return(m)

choropleth_assessed_val_by_year(2019)